In [9]:
# Imports
from random import randint
from abc import ABC, abstractmethod
from dataclasses import dataclass
from typing import Generic, TypeVar
import statistics


In [10]:
# Naive method
def six_sided():
    # distributions hard coded
    return randint(1, 6)

print(six_sided())

def roll_dice():
    # can only sample in a limited manner
    return six_sided() + six_sided() 

print(roll_dice())

5
4


In [11]:
# Using a class to define an interface
class Distribution(ABC):
    @abstractmethod
    def sample(self):
        pass

# Use the abstract "Distribution" class to generate a concrete class "Die"
class Die(Distribution):
    # Initialize as a uniform over n=sides
    def __init__(self, sides):
        self.sides = sides
    # Define attribute "sample" that returns realization of Die
    def sample(self):
        return random.randint(1, self.sides)
    # In a class - can choose what calling the class returns
    def __repr__(self):
        return f"Die(sides={self.sides})" 
    # Easier bugfixes and removes errors in comparisons to other objects
        # Returns self as function definition
    def __eq__(self, other):
        if isinstance(other, Die):
            return self.sides == other.sides
        return False



# Generate class
six_sided = Die(6)

# Sample using class
print(six_sided.sample())

# Generate function to roll 2 dice
def roll_dice():
    return six_sided.sample() + six_sided.sample()
print(roll_dice())

# Changed due to __repr__ in class definition for debugging
print(six_sided)

# Comparison of different instances of the class
print(six_sided==six_sided)
# Changed due to __eq__ in class definition
print(six_sided==Die(6))
print(Die(6)==Die(6)) 
print(Die(6)==None)

3
9
Die(sides=6)
True
True
True
False


In [12]:
# Defining all these things is tedious
# Use decorator for dataclass to avoid all of this - frozen prevents changing parameters
@dataclass (frozen=True)
class Die(Distribution):
    # Restricts parameter sides: to be an int
    sides: int
    # Defines the sample attribute
    def sample(self):
        return random.randint(1, self.sides)
    
# Generate class
six_sided = Die(6)

# Sample using class
print(six_sided.sample())

# Generate function to roll 2 dice
def roll_dice():
    return six_sided.sample() + six_sided.sample()
print(roll_dice())

# Changed due to __repr__ in class definition for debugging
print(six_sided)

# Comparison of different instances of the class
print(six_sided==six_sided)
# Changed due to __eq__ in class definition
print(six_sided==Die(6))
print(Die(6)==Die(6)) 
print(Die(6)==None)

# Changed due to frozen - raises error
# six_sided.sides = 10

# Instead we must generate a new copy - easy with replace from dataclass
d20 = dataclasses.replace(six_sided, sides=20)
print(d20)
# Finally - immutable objects from frozen lets us use immutable objects as dict keys and set elements
    # Essentially function as plain data - and not function references

3
4
Die(sides=6)
True
True
True
False
Die(sides=20)


In [21]:
# Important to add type annotations - but sometimes unclear what the type is

# A type variable named ”A”
A = TypeVar("A")
            
# Distribution is ”generic in A”
class Distribution(ABC, Generic[A]):
    # Sampling must produce a value of type A
    @abstractmethod
    def sample(self) -> A:
        pass

# Can now specify what type of Distribution we generate
@dataclass (frozen=True)
class Die(Distribution[int]):
    # Defines the sample attribute
    def sample(self):
        return random.randint(1, self.sides)
    
def expected_value(d: Distribution[float], n: int = 100) -> float:
    return statistics.mean(d.sample() for _ in range(n))

